### Segmenting and Clustering Neighbourhoods in Toronto Week 3 Part 1 Assigment

#### Import all necessary libraries

In [ ]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import urllib.request
from urllib.request import urlopen
import json # library to handle JSON files
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [2]:
# install BeautifulSoup
!pip install bs4

     |████████████████████████████████| 102kB 16.5MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [3]:
# Install lxml
!pip install lxml

     |████████████████████████████████| 5.8MB 30.4MB/s eta 0:00:01


In [4]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [6]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup 

In [7]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [8]:
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

##### Bring back ALL of the tables.We know the data resides within an HTML table so firstly we send Beautiful Soup off to retrieve
##### all instances of the table tag within the page and add them to an array called all_tables

In [9]:
all_tables=soup.find_all("table")

In [10]:
right_table=soup.find('table', class_='wikitable sortable')

In [11]:
# Loop through the rows.
A=[]
B=[]
C=[]

for row in right_table.find_all('tr'):
    cells=row.find_all('td')
    states=row.find_all('a') #To store second and third columns data if there are links
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        #blnk=cells[1].find_all('a')
        B.append(cells[1].find(text=True))
        #nlnk=cells[2].find_all('a')
        C.append(cells[2].find(text=True))

In [12]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned\n
8,M8A,Not assigned,Not assigned\n
9,M9A,Downtown Toronto,Queen's Park


### Instructions

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 7th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [13]:
# Make a list with the Bouroughs with Not assigned
# Remove them from the df
# Reindex the df

In [14]:
# replace "Not Assigned" to NaN
df.replace("Not assigned", np.nan, inplace = True)
df.replace("Not assigned\n", np.nan, inplace = True)
df = df.replace('\n',' ', regex=True) # Remove new line caracter
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,NaN
8,M8A,NaN,NaN
9,M9A,Downtown Toronto,Queen's Park


In [15]:
# Drop all rows with NaN in Borough Column
# simply drop whole row with NaN in "price" column
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped two rows
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,NaN
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [16]:
df.to_csv('extracted_df_toronto.csv')

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 7th cell in the table on the Wikipedia page,the value of the Borough and the Neighborhood columns will be Queen's Park.

#### Find Neighbourhood index with NaN and replace with the value in Borough "ffill" is a method that is used with fillna function to forward fill the values in a dataframe.  So if there is a NaN cell then ffill will replace that NaN value with the next row or column based on the axis 0 or 1 that you choose

In [ ]:
df1 = df.ffill(axis=1)
df1.head(10)

In [19]:
# Groups Neighbourd by repeated Postcode
df2 = df1.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))

In [20]:
df2.reset_index(level=['Postcode','Borough'], inplace=True) 
df2.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson ,Garden District"


In [21]:
df2.to_csv('extracted_df_toronto_df2.csv')

In [23]:
print("The number of rows for the dataframe is: ", df2.shape[0])

The number of rows for the dataframe is:  103
